In [9]:
import importlib
import sys
importlib.reload(sys.modules['Module'])
from Module import CourseModule
from scrape_helper import get_module_links, get_courses

In [10]:
# Get the list of all courses from the given url
courses = get_courses("https://hpi.de/studium/im-studium/lehrveranstaltungen.html")

# Dictionary to store the courses and their modules (trimmed links)
course_modules = {}

# Dictionary to store trimmed and full module links
module_links_dict = {}

for course in courses:

    # get the list of modules for the given course
    module_links = get_module_links(course)

    # If no modules were found, skip the course
    if module_links == []:
        print("No modules found for course:", course)
        continue

    # Cut the host part of the url
    module_links_trimmed = [link.split("/")[-1] for link in module_links]

    # add the trimmed and full links to the dictionary
    for trimmed, full in zip(module_links_trimmed, module_links):
        module_links_dict[trimmed] = full
    
    # add the course and its modules to the dictionary
    course_modules[course] = module_links_trimmed


Div with class 'tx-ciuniversity' not found for url: https://hpi.de//studium/im-studium/lehrveranstaltungen/professional-skills.html
No modules found for course: https://hpi.de//studium/im-studium/lehrveranstaltungen/professional-skills.html
Div with class 'tx-ciuniversity' not found for url: https://hpi.de/entrepreneurship/home.html
No modules found for course: https://hpi.de/entrepreneurship/home.html


In [73]:
# create a list of modules of class Module
modules = {}

# create a Module object for each module
for i, (url_trimmed, url) in enumerate(module_links_dict.items()):
    if i%10 == 0:
        print(f"Fetching data for module {i+1}/{len(module_links_dict)}")

    # create a Module object for the given url
    module = CourseModule(url)
    module.get_landing_page_information()
    module.get_evaluation_metrics()

    # add the module to the list of modules
    modules[url_trimmed] = module

    if i == 20:
        break

Fetching data for module 1/115


AttributeError: module 're' has no attribute 'find'

In [ ]:
for x, y in modules.items():
    if y._website_url:
        print(y._website_url) 

/friedrich/teaching/ws23/algogems.html
/friedrich/teaching/ws23/algoriddles.html
/friedrich/teaching/ws23/swiss-system-tournaments.html
https://moodle.hpi.de/course/view.php?id=658
/friedrich/teaching/ws23/competitive2.html
/naumann/teaching/current-courses/ws-23-24/datenbanksysteme-ii-vl-bachelor.html
https://moodle.hpi.de/course/view.php?id=656
/friedrich/teaching/ws23/mathe1.html
/friedrich/teaching/ws23/probtheory.html
/friedrich/teaching/ws23/ti1.html
/friedrich/teaching/ws23/tiunplugged.html
https://hpi.de/entrepreneurship/understanding-entrepreneurship.html
https://hpi.de/entrepreneurship/wirtschaftliche-grundlagen.html
/friedrich/teaching/ws23/competitive2.html
/naumann/teaching/current-courses/ws-23-24/advanced-data-profiling.html
/friedrich/teaching/ws23/probtheory.html
https://hpi.de/entrepreneurship/ai-in-practice.html
https://hpi.de/baudisch/dokuwiki/algorithmic-folding-ws2024/start
https://moodle.hpi.de/course/view.php?id=689
/rabl/teaching/winter-term-2023-24/big-data-la

In [ ]:
test = Module(list(modules.values())[0]._url)

In [69]:
list(modules.values())[2].__dict__

{'_url': 'https://hpi.de/studium/im-studium/lehrveranstaltungen/it-systems-engineering-ma/lehrveranstaltung/wise-23-24-3925-advanced-track-design-thinking-d_school.html',
 '_url_trimmed': 'wise-23-24-3925-advanced-track-design-thinking-d_school.html',
 '_title': 'Advanced Track Design Thinking (D-School) (Wintersemester 2023/2024)',
 '_description': None,
 '_dates': 'This hybrid course will be held weekly on two program days (Monday and Thursday) from 9:00 am to 3:00 pm (CET).\nThe dates for the winter semester 22/23\n- First day of the program: 17.10.2022 (as pre-launch)\n- Last day of the program: 16.02.2023\n- Deadline for submission of the final version of the project documentation: 06.03.2023\n\nThe\xa0total time investment\xa0in this course is\xa016-20 hours/week. On program days, we require active participation for a total of 10 working hours (either onsite or virtual). In addition, we expect all participants to invest an additional 6-10 working hours per week in teamwork and in

In [71]:
list(modules.values())[4].__dict__

{'_url': 'https://hpi.de/studium/im-studium/lehrveranstaltungen/it-systems-engineering-ba/lehrveranstaltung/wise-23-24-3902-algorithmic-problem-solving.html',
 '_url_trimmed': 'wise-23-24-3902-algorithmic-problem-solving.html',
 '_title': 'Algorithmic Problem Solving (Wintersemester 2023/2024)',
 '_description': None,
 '_dates': None,
 '_rooms': None,
 '_lecturers': ['Dr. Timo Kötzing',
  '(Algorithm Engineering)',
  'Dr. Pascal Lenzner',
  '(Algorithm Engineering)'],
 '_evaluation_metrics': None,
 '_general_info': {'Semesterwochenstunden': '2',
  'ECTS': '3',
  'Benotet': 'Ja',
  'Einschreibefrist': '01.10.2023 - 31.10.2023',
  'Lehrform': 'Vorlesung / Übung',
  'Belegungsart': 'Wahlpflichtmodul',
  'Lehrsprache': 'Deutsch'},
 'soup': <div class="tx-ciuniversity-course">
 <h1>Algorithmic Problem Solving (Wintersemester 2023/2024)</h1>
 		Dozent: <i><a href="https://hpi.de/friedrich/people/timo-koetzing.html">Dr. Timo Kötzing</a>
 <a href="https://hpi.de/friedrich/home.html">(Algorithm

In [66]:
module = CourseModule("https://hpi.de/studium/im-studium/lehrveranstaltungen/it-systems-engineering-ma/lehrveranstaltung/wise-23-24-3925-advanced-track-design-thinking-d_school.html")
module.get_landing_page_information()

In [56]:
module.__dict__

{'_url': 'https://hpi.de/studium/im-studium/lehrveranstaltungen/it-systems-engineering-ma/lehrveranstaltung/wise-23-24-3925-advanced-track-design-thinking-d_school.html',
 '_url_trimmed': 'wise-23-24-3925-advanced-track-design-thinking-d_school.html',
 '_title': 'Advanced Track Design Thinking (D-School) (Wintersemester 2023/2024)',
 '_description': None,
 '_dates': 'This hybrid course will be held weekly on two program days (Monday and Thursday) from 9:00 am to 3:00 pm (CET).\nThe dates for the winter semester 22/23\n- First day of the program: 17.10.2022 (as pre-launch)\n- Last day of the program: 16.02.2023\n- Deadline for submission of the final version of the project documentation: 06.03.2023\n\nThe\xa0total time investment\xa0in this course is\xa016-20 hours/week. On program days, we require active participation for a total of 10 working hours (either onsite or virtual). In addition, we expect all participants to invest an additional 6-10 working hours per week in teamwork and in